In [1]:
#Import pandas and read the dataset
import pandas as pd
col_names = ['ID', 'Limit_Bal', 'Gender', 'Education_lvl', 'Mar_status', 'Age', 'Pay_Level', 'Pay_Aug', 'Pay_July', 'Pay_June',
            'Pay_May', 'Pay_April', 'Bill_amt_Sept', 'Bill_amt_Aug', 'Bill_amt_July', 'Bill_amt_June',
             'Bill_amt_May', 'Bill_amt_April', 'Pay_amt_Sept', 'Pay_amt_Aug', 'Pay_amt_July', 'Pay_amt_June', 'Pay_amt_May',
            'Pay_amt_April', 'Default_status']

data = pd.read_csv('/home/nkiru/Documents/default_of_credit_card_clients.csv', names = col_names, header = 0, index_col = 'ID')



In [2]:
#Define a wrangle function
import numpy as np
def wrangle(X):
    X = X.copy()
    #Change the category name to match what is expected
    X['Mar_status'] = X['Mar_status'].map({0: 'others', 1:'married', 2: 'single', 3: 'others'}) 
    X['Education_lvl'] = X['Education_lvl'].map({1: 'graduate school', 2:'university', 3: 'high school',
                                                  4: 'others', 0:'others', 6:'others'})
    
    #Create new features from the Amount of bill statement from September to April
    bill_cols = ['Bill_amt_Sept', 'Bill_amt_Aug', 'Bill_amt_July', 'Bill_amt_June', 
                 'Bill_amt_June', 'Bill_amt_May', 'Bill_amt_April']
    pay_cols = ['Pay_amt_Sept', 'Pay_amt_Aug', 'Pay_amt_July', 'Pay_amt_June',
                'Pay_amt_May', 'Pay_amt_April']
    
    X['Total_bill'] = X[bill_cols].sum(axis = 1)
    X['Total_paid'] = X[pay_cols].sum(axis = 1)
    
    #Let's find difference betweenn what a customer has spent and what they have paid

    X['Bal'] = (X['Limit_Bal'] - X['Total_paid']) - (X['Limit_Bal'] - X['Total_bill'])
    
    #drop bill_cols and pay_cols to avoid data leakage
    X = X.drop(columns = bill_cols)
    X = X.drop(columns = pay_cols)

    
    return X
#aply thr wrangle function on the dataset
df = wrangle(data)

In [3]:
target = 'Default_status'
features = ['Limit_Bal', 'Total_bill', 'Total_paid', 'Pay_Level','Age', 'Education_lvl']
X = df[features]
y = df[target]
#import train_test_split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42)
print('X_train shape:', X_train.shape, 'X_test shape:', X_test.shape, 'y_train shape:', y_train.shape, 'y_test:', y_test.shape)

X_train shape: (24000, 6) X_test shape: (6000, 6) y_train shape: (24000,) y_test: (6000,)


In [ ]:
from scipy.stats import randint, uniform
from sklearn.pipeline import make_pipeline
from category_encoders import OrdinalEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier

#from sklearn.model_selection import RandomizedSearchCV

best_xgb = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy = 'mean'),
    XGBClassifier(
        learning_rate = 0.01,
        max_depth = None,
        n_estimators = 512,
        scale_pos_weight = 1,
        random_state = 42,
        n_jobs = -1)
)

best_xgb.fit(X_train, y_train);

/home/nkiru/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[20:12:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [ ]:
def predict(limit_bal, pay_level, total_bill, total_paid,  education_lvl):
    df = pd.DataFrame(
        columns  = ['Limit_Bal', 'Pay_Level','Age', 'Total_bill', 'Total_paid',  'Education_lvl'],
        data = [[limit_bal, pay_level, age, total_bill, total_paid,  education_lvl]]
    )

    y_pred = best_xgb.predict(df)[0]
    return f'The model predicted thet the cutomer has {y_pred:} default'


In [ ]:
from joblib import dump
dump(best_xgb, 'best_xgb.joblib', compress=True)

import joblib
import sklearn
import category_encoders as ce
import xgboost
print(f'joblib=={joblib.__version__}')
print(f'scikit-learn=={sklearn.__version__}')
print(f'category_encoders=={ce.__version__}')
print(f'xgboost=={xgboost.__version__}')